In [8]:
import pandas as pd
import os

folder = "experiments/"

paths = os.listdir(folder)
survey_paths = {path.split("_")[1] : path for path in paths if "survey" in path}
experiment_results_paths = {path.split("_")[1].split('.')[0] : path for path in paths if not "survey" in path}
experiment_ids = list(experiment_results_paths.keys())

print(survey_paths)

{'1651173731': 'comp1_1651173731_survey.csv', '1651174140': 'comp1_1651174140_survey.csv', '1651174244': 'comp1_1651174244_survey.csv', '1651174316': 'comp1_1651174316_survey.csv', '1651174376': 'comp1_1651174376_survey.csv', '1651174477': 'comp1_1651174477_survey.csv', '1651176011': 'comp1_1651176011_survey.csv', '1651176322': 'comp1_1651176322_survey.csv', '1651176556': 'comp1_1651176556_survey.csv', '1651176896': 'comp1_1651176896_survey.csv', '1651177208': 'comp1_1651177208_survey.csv', '1651177349': 'comp1_1651177349_survey.csv', '1651177412': 'comp1_1651177412_survey.csv', '1651177469': 'comp1_1651177469_survey.csv', '1651177572': 'comp1_1651177572_survey.csv', '1651177648': 'comp1_1651177648_survey.csv', '1651177694': 'comp1_1651177694_survey.csv', '1651177751': 'comp1_1651177751_survey.csv', '1651177890': 'comp1_1651177890_survey.csv', '1651178138': 'comp1_1651178138_survey.csv', '1651178210': 'comp1_1651178210_survey.csv', '1651178312': 'comp1_1651178312_survey.csv', '16511783

In [16]:

# Put all surveys and results into separate dictionaries
surveys = {}
results = {}
for id in experiment_ids:
    survey = None
    if id in survey_paths:
        survey_path = survey_paths[id]
        df_survey = pd.read_csv('experiments/' + survey_path)
        surveys.update({id : df_survey})
    if id in experiment_ids:
        results_path = experiment_results_paths[id]
        df_results = pd.read_csv('experiments/' + results_path)
        results.update({id : df_results})

    

# Do some cleaning of dfs

# TODO Make the time between clicks about differences NOT about position of t
# TODO Drop first column with the indexes
# TODO compute accuracy % based on distance from radius of circle (if x <= r, 100%; if x > r, r/x)



In [17]:
# Analysis of results first (then do surveys)

# Combine all dfs into one df
combined_results_df = pd.concat(results)

{'1651173731':    Unnamed: 0  circle_x  circle_y  circle_radius  click_x  click_y  \
 0           0       554       658             96      551      655   
 1           1      1025       704             88     1025      701   
 2           2       184       544             73      207      560   
 3           3       741       160             49      720      164   
 4           4        56       181             50       59      191   
 5           5       930       336             19      946      325   
 6           6       781       411             33      797      425   
 7           7       706       325             93      735      363   
 8           8       411        81             47      367      112   
 9           9      1121       466             88     1067      471   
 
    time_clicked  click_distance_from_circle_center  click_success  \
 0      1.486618                           4.242641           True   
 1      2.970781                           3.000000           T